# Machine Learning Algorithms

### Part 1: Brief Introduction to Linear Regression
---------------------------------------------------

Linear regression attempts to model the relationship between two variables by fitting a linear equation to observed data. One variable is considered to be an explanatory variable, and the other is considered to be a dependent variable. For example, a modeler might want to relate the weights of individuals to their heights using a linear regression model.

Before attempting to fit a linear model to observed data, a modeler should first determine whether or not there is a relationship between the variables of interest. This does not necessarily imply that one variable causes the other (for example, higher SAT scores do not cause higher college grades), but that there is some significant association between the two variables.

A linear regression line has an equation of the form Y = a + bX, where X is the explanatory variable and Y is the dependent variable. The slope of the line is b, and a is the intercept (the value of y when x = 0).
[1]

<img src="https://upload.wikimedia.org/wikipedia/commons/3/3a/Linear_regression.svg">

Example of simple linear regression, which has one independent variable [2]

#### Least-Squares Regression
The most common method for fitting a regression line is the method of least-squares. This method calculates the best-fitting line for the observed data by minimizing the sum of the squares of the vertical deviations from each data point to the line (if a point lies on the fitted line exactly, then its vertical deviation is 0). Because the deviations are first squared, then summed, there are no cancellations between positive and negative values.

<img src="http://www.stat.yale.edu/Courses/1997-98/101/lsline.gif">

#### Outliers and Influential Observations
After a regression line has been computed for a group of data, a point which lies far from the line (and thus has a large residual value) is known as an outlier. Such points may represent erroneous data, or may indicate a poorly fitting regression line. If a point lies far from the other data in the horizontal direction, it is known as an influential observation. The reason for this distinction is that these points have may have a significant impact on the slope of the regression line.

<img src="http://www.stat.yale.edu/Courses/1997-98/101/lsline2.gif">

#### Residuals
Once a regression model has been fit to a group of data, examination of the residuals (the deviations from the fitted line to the observed values) allows the modeler to investigate the validity of his or her assumption that a linear relationship exists. Plotting the residuals on the y-axis against the explanatory variable on the x-axis reveals any possible non-linear relationship among the variables, or might alert the modeler to investigate lurking variables.

<img src="http://www.stat.yale.edu/Courses/1997-98/101/lsresid.gif">

[Read More: Linear Regression](http://www.stat.yale.edu/Courses/1997-98/101/linreg.htm "Linear Regression : Yale")

[Read More: Linear Regression Course](https://onlinecourses.science.psu.edu/stat501/node/250 )

## Part 2 : Implementing Linear Regression
-------------------------

Getting the necessary libraries to get the work started.

In [1]:
import json
import pandas as pd
%matplotlib inline 
import matplotlib.pyplot as plt
import numpy as np                  #It provides some advance math functionalities to python
import pandas as pd                 #to load the data file as a Pandas data frame and analyze the data
from scipy import stats             #SciPy contains modules for optimization, linear algebra, integration, interpolation, special functions, FFT, signal and image processing, ODE solvers and other tasks common in science and engineering.
import seaborn as sns               #Seaborn is a Python visualization library based on matplotlib. It provides a high-level interface for drawing attractive statistical graphics.
import warnings
import random
from sklearn import cross_validation                                                        # Scikit-learn (formerly scikits.learn) is a free software machine learning library for the Python programming language.
from sklearn.cross_validation import KFold, cross_val_score, train_test_split               # It features various classification, regression and clustering algorithms including support vector machines, random forests, gradient boosting, k-means and DBSCAN,
from sklearn import metrics                                                                 # and is designed to interoperate with the Python numerical and scientific libraries NumPy and SciPy.
from datetime import datetime       #The datetime module supplies classes for manipulating dates and times in both simple and complex ways.
random.seed(datetime.now())
warnings.filterwarnings('ignore')


plt.rcParams['figure.figsize'] = (20, 10) #Defining size of plots
from sklearn import metrics
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing, cross_validation, svm
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.metrics import roc_curve # ROC Curves
from sklearn.metrics import auc # Calculating AUC(Area under the curve) for ROC's(Reciever Operating Characteristics)!
import warnings

warnings.filterwarnings('ignore')

C:\Users\rahil\Anaconda3\envs\py35\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


We get the data in CSV (Comma Seperated Value) Format, after data scrapping and cleaning. To read more on it : [Data Retrival](https://cocalc.com/projects/555ac5a9-25b4-4679-831c-1287ecda54c7/files/Blog/Blog.ipynb?session=default) and [Data Cleaning](https://github.com/MandyYang86/Ride-Optimization/blob/master/Data%20Clean%20Part/DataClean_OneMonth_Uber_Pool.ipynb).

So we need to import it in our pandas dataframe for further analysis.

In [2]:
df = pd.read_csv("uber_lyft_March.csv")
ds = df

Lets have a look at how our dataset looks.

In [3]:
df.head()

,date_time,uber_distance,uber_duration,uber_estimate,uber_high_estimate,uber_low_estimate,main_temp,weather,uber_price_per_second,lyft_distance,lyft_duration,lyft_max_estimate,lyft_min_estimate,lyft_estimate,lyft_price_per_second,average_duration
0,3/1/18 0:00,1.73,360,7.5,9,6,46.69,Rain,0.019380,1.76,414,3.40,3.40,3.40,0.008786,387.0
1,3/1/18 0:01,1.90,480,5.5,7,4,46.69,Rain,0.010816,1.79,537,3.53,3.53,3.53,0.006942,508.5
2,3/1/18 0:02,2.26,420,7.0,9,5,46.56,Rain,0.015436,2.20,487,3.85,3.85,3.85,0.008490,453.5
3,3/1/18 0:03,1.63,360,7.5,9,6,46.56,Rain,0.018029,1.70,472,3.46,3.46,3.46,0.008317,416.0
4,3/1/18 0:04,2.17,480,9.5,11,8,46.56,Rain,0.018393,2.21,553,3.47,3.47,3.47,0.006718,516.5


So we are only using the essential columns required, which will be the features that will help us predict the outcome. Here the date time will help us in indexing the dataframe for ease of access, whereas, other columns will be predictors, that will allow us to forecast the future using the historical data we have until now.

In [4]:
df.columns


Index(['date_time', 'uber_distance', 'uber_duration', 'uber_estimate',
       'uber_high_estimate', 'uber_low_estimate', 'main_temp', 'weather',
       'uber_price_per_second', 'lyft_distance', 'lyft_duration',
       'lyft_max_estimate', 'lyft_min_estimate', 'lyft_estimate',
       'lyft_price_per_second', 'average_duration'],
      dtype='object')

We need to check whether all the columns have proper data type we need for linear regression.

In [5]:
df.dtypes


date_time                 object
uber_distance            float64
uber_duration              int64
uber_estimate            float64
uber_high_estimate         int64
uber_low_estimate          int64
main_temp                float64
weather                   object
uber_price_per_second    float64
lyft_distance            float64
lyft_duration              int64
lyft_max_estimate        float64
lyft_min_estimate        float64
lyft_estimate            float64
lyft_price_per_second    float64
average_duration         float64
dtype: object

We find out that the weather column needs to converted into some kind of a numerical format as linear regression does not process categorical data. So we convert unique weather names and  assign it a numerical label, as a work around for using linear regression.

In [6]:
df[['weather']] = df["weather"].astype('category')

In [7]:
df['weather_label'] = df["weather"].cat.codes

So here we can see that which label represents which weather condition.

In [8]:
df.groupby('weather')['weather_label'].unique()

weather
Clear      [0]
Clouds     [1]
Drizzle    [2]
Fog        [3]
Haze       [4]
Mist       [5]
Rain       [6]
Snow       [7]
Name: weather_label, dtype: object

We drop unnecessary columns which will not be used in any further analysis.

In [9]:
df.drop(['uber_high_estimate', 'uber_low_estimate', 'lyft_max_estimate', 'lyft_min_estimate'], axis=1, inplace=True)

We will use data_time as our index now.

In [10]:
df.set_index('date_time', inplace=True)

In [11]:
df.head()

,uber_distance,uber_duration,uber_estimate,main_temp,weather,uber_price_per_second,lyft_distance,lyft_duration,lyft_estimate,lyft_price_per_second,average_duration,weather_label
date_time,,,,,,,,,,,,
3/1/18 0:00,1.73,360,7.5,46.69,Rain,0.019380,1.76,414,3.40,0.008786,387.0,6
3/1/18 0:01,1.90,480,5.5,46.69,Rain,0.010816,1.79,537,3.53,0.006942,508.5,6
3/1/18 0:02,2.26,420,7.0,46.56,Rain,0.015436,2.20,487,3.85,0.008490,453.5,6
3/1/18 0:03,1.63,360,7.5,46.56,Rain,0.018029,1.70,472,3.46,0.008317,416.0,6
3/1/18 0:04,2.17,480,9.5,46.56,Rain,0.018393,2.21,553,3.47,0.006718,516.5,6


What are the features and what is the label? We're trying to predict the price per second, so is that the label? If so, what are the featuers? When it comes to forecasting out the price, our label, the thing we're hoping to predict, is actually the future price. As such, our features are actually: uber_distance, uber_duration, lyft_price_per_second, average_duration

In [12]:
forecast_col = 'uber_price_per_second'
forecast_col_1 = 'lyft_price_per_second'

In [13]:
df['label'] = df[forecast_col]
df['label_1'] = df[forecast_col_1]

Generally, you want your features in machine learning to be in a range of -1 to 1. This may do nothing, but it usually speeds up processing and can also help with accuracy. Because this range is so popularly used, it is included in the preprocessing module of Scikit-Learn. To utilize this, you can apply preprocessing.scale to your X variable

In [14]:
X = np.array(df[['uber_distance','uber_duration','lyft_price_per_second','average_duration','weather_label']])
X = preprocessing.scale(X)

X1 = np.array(df[['lyft_distance','lyft_duration','uber_price_per_second','average_duration','weather_label']])
X1 = preprocessing.scale(X1)

Now comes the training and testing. The way this works is you take, for example, 75% of your data, and use this to train the machine learning classifier. Then you take the remaining 25% of your data, and test the classifier. Since this is your sample data, you should have the features and known labels. Thus, if you test on the last 25% of your data, you can get a sort of accuracy and reliability, often called the confidence score. There are many ways to do this, but, probably the best way is using the build in cross_validation provided, since this also shuffles your data for you. 


In [15]:
y = np.array(df['uber_price_per_second'])
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.25)

y1 = np.array(df['lyft_price_per_second'])
X1_train, X1_test, y1_train, y1_test = cross_validation.train_test_split(X1, y1, test_size=0.25)

The return here is the training set of features, testing set of features, training set of labels, and testing set of labels. Now, we're ready to define our classifier. There are many classifiers in general available through Scikit-Learn, and even a few specifically for regression.

In [16]:
clf = LinearRegression()
results = clf.fit(X_train, y_train)
accuracy = clf.score(X_test, y_test)
print(accuracy)

results = clf.fit(X1_train, y1_train)
accuracy = clf.score(X1_test, y1_test)
print(accuracy)

0.46567509338070623
0.27785464216072675


We can see that analysing the historical data and its features using linear regression doesnt give us favourable results. Lets see how we can analyse the errors to understand this better.

In [17]:
print (X.shape)
print (y.shape)
print (X_train.shape)
print (y_train.shape)
print (X_test.shape)
print (y_test.shape)

(44434, 5)
(44434,)
(33325, 5)
(33325,)
(11109, 5)
(11109,)


In [18]:
# Fit the linear model
model = linear_model.LinearRegression()
results = model.fit(X_train, y_train)

# Print the coefficients
print (results.intercept_, results.coef_)

0.014779471244987737 [ 0.00170932  0.00055261  0.00190103 -0.00348194  0.00020052]


Lets start with the errors in prediction of training sets.

In [19]:
uber_pred = model.predict(X_test) # Predicting the features of the dataset

In [20]:
output = pd.DataFrame()                     # Create a blank dataframe
output['uber_pred'] = uber_pred             # Add a column of predicted value of uber prices
output['actual'] = y_test                   # Add a column of the actual value of uber prices, we put them side by side for easy comparison
output['percent_linear_regression_error'] = abs(output['actual']-output['uber_pred'])*100/output['actual']              # Finding the precentage error between the predicted values and the actual values
train_mean = np.mean(y_train)                #Baseline prediction - is the average value of dependent variable. So we are taking a mean baseline apporach.
output['baseline_error'] = abs(output['actual']-train_mean)*100/output['actual']
output.head(n=50)

,uber_pred,actual,percent_linear_regression_error,baseline_error
0,0.016814,0.017316,2.898004,14.548791
1,0.013113,0.009590,36.729357,54.289710
2,0.016005,0.015931,0.460145,7.121967
3,0.016263,0.012799,27.065443,15.611906
4,0.010821,0.009040,19.693094,63.674769
5,0.014911,0.013052,14.240990,13.365836
6,0.017691,0.019200,7.858594,22.933616
7,0.014153,0.013798,2.575454,7.237467
8,0.014036,0.012452,12.720393,18.829254
9,0.012042,0.008398,43.388628,76.186973


We can see that the regression error is much less compared to the baseline error, but still significant enough to decrease the accuracy of our prediction.

In [21]:
print ('Mean Baseline Error: ', output['baseline_error'].mean())
print ('Mean Regression Error: ', output['percent_linear_regression_error'].mean())

Mean Baseline Error:  27.554613385672802
Mean Regression Error:  18.707245355226576


###### Random Forest Regression

A random forest is a meta estimator that fits a number of classifying decision trees on various sub-samples of the dataset and use averaging to improve the predictive accuracy and control over-fitting.

The Random Forest solves the instability problem using bagging. We simply estimate the desired Regression Tree on many bootstrap samples (re-sample the data many times with replacement and re-estimate the model) and make the final prediction as the average of the predictions across the trees.

There is one small (but important) detail to add. The Random Forest adds a new source of instability to the individual trees. Every time we calculate a new optimal variable-observation point to split the tree, we do not use all variables. Instead, we randomly select 2/3 of the variables. This will make the individual trees even more unstable, but as I mentioned here, bagging benefits from instability.

[Read More: Random Forest Regression](https://www.r-bloggers.com/how-random-forests-improve-simple-regression-trees/)

In [22]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators = 1000, random_state = 18, max_depth = 5) # We chose 1000 trees in the forest, 18 is the seed for the forest and we chose the maximumn depth of the trees to be 5.
ouput_random_forest = model.fit(X_train, y_train)
random_forest_predictions = model.predict(X_test)

In [23]:
output['uber_pred_rf'] = random_forest_predictions                                   # Adding the random forest predictions to the output dataframe
output['percent_random_forest_error'] = abs(output['actual']-output['uber_pred_rf'])*100/output['actual']                  # Calculating the percentage error between the actual and the random forest predicted values.
output.head(n=50)

,uber_pred,actual,percent_linear_regression_error,baseline_error,uber_pred_rf,percent_random_forest_error
0,0.016814,0.017316,2.898004,14.548791,0.015533,10.299190
1,0.013113,0.009590,36.729357,54.289710,0.013129,36.898197
2,0.016005,0.015931,0.460145,7.121967,0.013178,17.284054
3,0.016263,0.012799,27.065443,15.611906,0.014937,16.706879
4,0.010821,0.009040,19.693094,63.674769,0.012361,36.731897
5,0.014911,0.013052,14.240990,13.365836,0.014569,11.621041
6,0.017691,0.019200,7.858594,22.933616,0.017877,6.892631
7,0.014153,0.013798,2.575454,7.237467,0.012870,6.729413
8,0.014036,0.012452,12.720393,18.829254,0.014705,18.095251
9,0.012042,0.008398,43.388628,76.186973,0.010619,26.438498


Not much difference is seen using this method, lets try something else.

In [24]:
print ('Mean Baseline Error: ', output['baseline_error'].mean())
print ('Mean Linear Regression Error: ', output['percent_linear_regression_error'].mean())
print ('Mean Random Forest Error: ', output['percent_random_forest_error'].mean())


Mean Baseline Error:  27.554613385672802
Mean Linear Regression Error:  18.707245355226576
Mean Random Forest Error:  18.450691144126264


###### Mean Squared Error

**The root mean squared error is more sensitive than other measures to the occasional large error**: the squaring process gives disproportionate weight to very large errors. If an occasional large error is not a problem in your decision situation (e.g., if the true cost of an error is roughly proportional to the size of the error, not the square of the error), then the MAE or MAPE may be a more relevant criterion. In many cases these statistics will vary in unison--the model that is best on one of them will also be better on the others--but this may not be the case when the error distribution has outliers. If one model is best on one measure and another is best on another measure, they are probably pretty similar in terms of their average errors. In such cases you probably should give more weight to some of the other criteria for comparing models--e.g., simplicity, intuitive reasonableness, etc. 

[Read More: Mean Squared Error](https://people.duke.edu/~rnau/compare.htm)

In [25]:
from sklearn.metrics import mean_squared_error
from math import sqrt 

ms_regression = mean_squared_error(output['actual'], output['uber_pred'])
ms_rf = mean_squared_error(output['actual'],output['uber_pred_rf'])

print ('MSE of linear regression: ', ms_regression)
print ('MSE of random forest: ', ms_rf)

rms_regression = sqrt(mean_squared_error(output['actual'], output['uber_pred']))
rms_rf = sqrt(mean_squared_error(output['actual'],output['uber_pred_rf']))

print ('RMSE of linear regression: ', rms_regression)
print ('RMSE of random forest: ', rms_rf)

MSE of linear regression:  1.3541549794823842e-05
MSE of random forest:  1.314561782367674e-05
RMSE of linear regression:  0.003679884481179245
RMSE of random forest:  0.0036256885999319827


In [26]:
import xgboost as xgb
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test)
param = {'max_depth':4, 'eta':1, 'silent':1, 'objective':'reg:linear', 'eval_metric': 'auc' }
num_round = 5
model = xgb.train(param, dtrain, num_round)
xgboost_predictions = model.predict(dtest)

In [27]:
output['uber_pred_xgboost'] = xgboost_predictions
output['percent_xgboost_error'] = abs(output['actual']-output['uber_pred_xgboost'])*100/output['actual']
output.head(n=50)

,uber_pred,actual,percent_linear_regression_error,baseline_error,uber_pred_rf,percent_random_forest_error,uber_pred_xgboost,percent_xgboost_error
0,0.016814,0.017316,2.898004,14.548791,0.015533,10.299190,0.016412,5.222478
1,0.013113,0.009590,36.729357,54.289710,0.013129,36.898197,0.012034,25.478998
2,0.016005,0.015931,0.460145,7.121967,0.013178,17.284054,0.013866,12.963954
3,0.016263,0.012799,27.065443,15.611906,0.014937,16.706879,0.016412,28.230018
4,0.010821,0.009040,19.693094,63.674769,0.012361,36.731897,0.011512,27.345168
5,0.014911,0.013052,14.240990,13.365836,0.014569,11.621041,0.013560,3.889981
6,0.017691,0.019200,7.858594,22.933616,0.017877,6.892631,0.018772,2.230532
7,0.014153,0.013798,2.575454,7.237467,0.012870,6.729413,0.012650,8.318537
8,0.014036,0.012452,12.720393,18.829254,0.014705,18.095251,0.013484,8.283314
9,0.012042,0.008398,43.388628,76.186973,0.010619,26.438498,0.009223,9.823300


In [28]:
print ('Mean Baseline Error: ', output['baseline_error'].mean())
print ('Mean Linear Regression Error: ', output['percent_linear_regression_error'].mean())
print ('Mean Random Forest Error: ', output['percent_random_forest_error'].mean())
print ('Mean XGBoost Error: ', output['percent_xgboost_error'].mean())


Mean Baseline Error:  27.554613385672802
Mean Linear Regression Error:  18.707245355226576
Mean Random Forest Error:  18.450691144126264
Mean XGBoost Error:  18.41060495973445


This shows us that even with other boosting algorithms we are unable to get the desired results. There is the possibility that the features we are using are not enough for predicting our label.

## References

[1] http://www.stat.yale.edu/Courses/1997-98/101/linreg.htm

[2] https://pythonprogramming.net/machine-learning-tutorial-python-introduction/

## License

The content of this project itself is licensed under the Creative Commons Attribution 3.0 license, and the underlying source code used to format and display that content is licensed under the [MIT License](https://github.com/rahilshah10/IS/blob/master/LICENSE).